In [3]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

import utils.data_processing_bronze_table
import utils.data_processing_silver_table
import utils.data_processing_gold_table

In [5]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 19:24:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/27 19:24:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
# Bronze / Silver / Gold row counts & schemas
for p in [
    "datamart/bronze/lms/",
    "datamart/silver/loan_daily/",
    "datamart/gold/label_store/",
]:
    print("\n===", p, "===")
    df = spark.read.parquet(p)
    print("rows:", df.count())
    df.printSchema()
    df.show(5, truncate=False)



=== datamart/bronze/lms/ ===


25/09/27 19:25:07 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:387)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:443)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:493)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:485)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scala:80)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.

Py4JJavaError: An error occurred while calling o26.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (88c6508da389 executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:387)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:443)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:493)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:485)
	at org.apache.spark.sql.execution.datas

In [7]:
import os
print([f for f in os.listdir("datamart/bronze/lms/")][:10])


['bronze_loan_daily_2023_01_01.csv', 'bronze_loan_daily_2023_02_01.csv', 'bronze_loan_daily_2023_03_01.csv', 'bronze_loan_daily_2023_04_01.csv', 'bronze_loan_daily_2023_05_01.csv', 'bronze_loan_daily_2023_06_01.csv', 'bronze_loan_daily_2023_07_01.csv', 'bronze_loan_daily_2023_08_01.csv', 'bronze_loan_daily_2023_09_01.csv', 'bronze_loan_daily_2023_10_01.csv']


In [8]:
from pyspark.sql import functions as F

# Bronze (CSV)
bronze = (spark.read
          .option("header", "true")
          .option("inferSchema", "true")
          .csv("datamart/bronze/lms/"))
print("bronze rows:", bronze.count()); bronze.printSchema()

# Silver (Parquet)
silver = spark.read.parquet("datamart/silver/loan_daily/")
print("silver rows:", silver.count()); silver.printSchema()

# Gold (Parquet)
gold = spark.read.parquet("datamart/gold/label_store/")
print("gold rows:", gold.count()); gold.printSchema()


bronze rows: 104288
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- loan_start_date: date (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- installment_num: integer (nullable = true)
 |-- loan_amt: integer (nullable = true)
 |-- due_amt: double (nullable = true)
 |-- paid_amt: double (nullable = true)
 |-- overdue_amt: double (nullable = true)
 |-- balance: double (nullable = true)
 |-- snapshot_date: date (nullable = true)



AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually.

In [9]:
import os, glob

print("Silver dir exists? ", os.path.exists("datamart/silver/loan_daily/"))
print("Silver contents: ", os.listdir("datamart/silver/loan_daily/")[:10])

print("Gold dir exists? ", os.path.exists("datamart/gold/label_store/"))
print("Gold contents: ", os.listdir("datamart/gold/label_store/")[:10])


Silver dir exists?  True
Silver contents:  ['silver_loan_daily_2023_01_01.parquet', 'silver_loan_daily_2023_02_01.parquet', 'silver_loan_daily_2023_03_01.parquet', 'silver_loan_daily_2023_04_01.parquet', 'silver_loan_daily_2023_05_01.parquet', 'silver_loan_daily_2023_06_01.parquet', 'silver_loan_daily_2023_07_01.parquet', 'silver_loan_daily_2023_08_01.parquet', 'silver_loan_daily_2023_09_01.parquet', 'silver_loan_daily_2023_10_01.parquet']
Gold dir exists?  True
Gold contents:  ['gold_label_store_2023_01_01.parquet', 'gold_label_store_2023_02_01.parquet', 'gold_label_store_2023_03_01.parquet', 'gold_label_store_2023_04_01.parquet', 'gold_label_store_2023_05_01.parquet', 'gold_label_store_2023_06_01.parquet', 'gold_label_store_2023_07_01.parquet', 'gold_label_store_2023_08_01.parquet', 'gold_label_store_2023_09_01.parquet', 'gold_label_store_2023_10_01.parquet']


In [10]:
# SILVER
silver = spark.read.parquet("datamart/silver/loan_daily/*.parquet")
print("silver rows:", silver.count()); silver.printSchema()

# GOLD
gold = spark.read.parquet("datamart/gold/label_store/*.parquet")
print("gold rows:", gold.count()); gold.printSchema()


silver rows: 104288
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- loan_start_date: date (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- installment_num: integer (nullable = true)
 |-- loan_amt: float (nullable = true)
 |-- due_amt: float (nullable = true)
 |-- paid_amt: float (nullable = true)
 |-- overdue_amt: float (nullable = true)
 |-- balance: float (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- mob: integer (nullable = true)
 |-- installments_missed: integer (nullable = true)
 |-- first_missed_date: date (nullable = true)
 |-- dpd: integer (nullable = true)

gold rows: 8974
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- label_def: string (nullable = true)
 |-- snapshot_date: date (nullable = true)



In [3]:
import pandas as pd
pd.read_csv("datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv")["snapshot_date"].unique()



array(['2023-01-01'], dtype=object)

In [4]:
pd.read_csv("datamart/bronze/features/clickstream/bronze_clickstream_2023_01_01.csv").head()

,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date,ingest_dt,snap_ym,source_file
0,63,118,80,121,55,193,111,112,-101,83,...,114,35,85,-73,76,CUS_0x1037,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
1,-108,182,123,4,-56,27,25,-6,284,222,...,35,130,94,111,75,CUS_0x1069,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
2,-13,8,87,166,214,-98,215,152,129,139,...,125,-130,354,17,302,CUS_0x114a,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
3,-85,45,200,89,128,54,76,51,61,139,...,163,37,207,180,118,CUS_0x1184,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
4,55,120,226,-86,253,97,107,68,103,126,...,159,-26,104,118,184,CUS_0x1297,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv


In [7]:
import glob
len(glob.glob("datamart/bronze/features/clickstream/bronze_clickstream_*.csv"))

24

In [8]:
len(glob.glob("datamart/bronze/lms/bronze_loan_daily_*.csv"))

24